In [1]:
import sympy as sy
from sympy.utilities.codegen import codegen

import os
from pathlib import Path


In [2]:
x0, y0, z0 = sy.symbols('x0, y0, z0')
dx0, dy0, dz0 = sy.symbols('dx0, dy0, dz0')

x = sy.Matrix([[x0, y0, z0]]).T
dx = sy.Matrix([[dx0, dy0, dz0]]).T

#x_norm = x.norm(ord = 2)
x_norm = (x0**2 + y0**2 + z0**2) ** (1/2)
x_hat = x / x_norm

print(x_norm)

(x0**2 + y0**2 + z0**2)**0.5


In [3]:
### アトらくと関連 ###

sigma_alpha, sigma_gamma, w_u, w_l, alpha, epsilon = sy.symbols('sigma_alpha, sigma_gamma, w_u, w_l, alpha, epsilon')


alpha_x = sy.exp(-((x_norm**2) / (2 * sigma_alpha ** 2)))
gamma_x = sy.exp(-((x_norm**2) / (2 * sigma_gamma ** 2)))
w_x = gamma_x * w_u + (1 - gamma_x) * w_l
nabla_x_phi = (1 - sy.exp(-2 * alpha * x_norm)) / (1 + sy.exp(-2 * alpha * x_norm)) * x_hat


M = w_x  * ((1 - alpha_x) * nabla_x_phi * nabla_x_phi.T + (alpha_x + epsilon) * sy.eye(3))

print(sy.simplify(M))

Matrix([[(-w_l*(1 - exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_gamma**2))) + w_u)*(epsilon*(exp(2*alpha*(x0**2 + y0**2 + z0**2)**0.5) + 1)**2*(x0**2 + y0**2 + z0**2)**1.0*exp((x0**2 + y0**2 + z0**2)**1.0/sigma_alpha**2) - x0**2*(1 - exp(2*alpha*(x0**2 + y0**2 + z0**2)**0.5))**2*(1 - exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_alpha**2)))*exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_alpha**2)) + (exp(2*alpha*(x0**2 + y0**2 + z0**2)**0.5) + 1)**2*(x0**2 + y0**2 + z0**2)**1.0*exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_alpha**2)))*exp(-(1/(2*sigma_gamma**2) + sigma_alpha**(-2))*(x0**2 + y0**2 + z0**2)**1.0)/((exp(2*alpha*(x0**2 + y0**2 + z0**2)**0.5) + 1)**2*(x0**2 + y0**2 + z0**2)**1.0), x0*y0*(w_l*(exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_gamma**2)) - 1) + w_u)*(exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_alpha**2)) - 1)*exp(-(sigma_gamma**(-2) + sigma_alpha**(-2))*(x0**2 + y0**2 + z0**2)**1.0/2)*tanh(alpha*(x0**2 + y0**2 + z0**2)**0.5)**2/(x0**2 + y0**2 + z0**2)**1.0, x0*z0*(w_l*(exp((x0**2 + y

In [4]:
# 曲率校xi_Mの前半を計算
partialx_m1 = M[:, 0:1].jacobian(x)
partialx_m2 = M[:, 1:2].jacobian(x)
partialx_m3 = M[:, 2:3].jacobian(x)

# print(sy.simplify(partialx_m1))

partialx_m1_dx = partialx_m1 * dx
partialx_m2_dx = partialx_m2 * dx
partialx_m3_dx = partialx_m3 * dx

xi_M_before_before = partialx_m1_dx.row_join(partialx_m2_dx)
xi_M_before_before = xi_M_before_before.row_join(partialx_m3_dx)

xi_M_before = xi_M_before_before * dx

#print(xi_M_before)

# 曲率校xi_Mの後半を計算
xi_M_after_before = dx.T * M * dx
xi_M_after = (xi_M_after_before.jacobian(x)).T #　sympyにはナブラあるかも？

# 合体
xi_M = xi_M_before + xi_M_after
xi_M = sy.simplify(xi_M)  # Elisじゃ無理
print(xi_M)

Matrix([[(dx0*(dx0*x0*(1.0*sigma_alpha**2*(w_l - w_u)*(exp(2*alpha*(x0**2 + y0**2 + z0**2)**0.5) + 1)*(x0**2 + y0**2 + z0**2)**7.0*(epsilon*(exp(2*alpha*(x0**2 + y0**2 + z0**2)**0.5) + 1)**2*(x0**2 + y0**2 + z0**2)**1.0*exp((x0**2 + y0**2 + z0**2)**1.0/sigma_alpha**2) - x0**2*(1 - exp(2*alpha*(x0**2 + y0**2 + z0**2)**0.5))**2*(1 - exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_alpha**2)))*exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_alpha**2)) + (exp(2*alpha*(x0**2 + y0**2 + z0**2)**0.5) + 1)**2*(x0**2 + y0**2 + z0**2)**1.0*exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_alpha**2)))*exp((sigma_alpha**2 + 6*sigma_gamma**2)*(x0**2 + y0**2 + z0**2)**1.0/(2*sigma_alpha**2*sigma_gamma**2)) + sigma_gamma**2*(-w_l*(1 - exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_gamma**2))) + w_u)*(x0**2 + y0**2 + z0**2)**1.0*(-4.0*alpha*sigma_alpha**2*x0**2*(1 - exp(2*alpha*(x0**2 + y0**2 + z0**2)**0.5))**2*(1 - exp((x0**2 + y0**2 + z0**2)**1.0/(2*sigma_alpha**2)))*(x0**2 + y0**2 + z0**2)**5.5 + 4.0*alpha*sigma_alpha**2*x0*

In [5]:
def gen(func, name):
    [(c_name, c_code), (h_name, c_header)] = codegen(
        name_expr=(name, func),
        language="C",
        project= name + "project",
        to_files=False
    )

    f = open(c_name, 'w')
    f.write(c_code)
    f.close()

    f = open(h_name, 'w')
    f.write(c_header)
    f.close()


gen(xi_M, "xi.py")